In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 

In [3]:
df = pd.read_csv("data/mongo_data.csv", index_col='ts')
df = df.drop('Unnamed: 0', 1)
#Remove cols with the same value

def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)

df = df.loc[:,np.invert(unique_cols(df))]
print(df.shape)
df.tail()

C:\Users\pedro\AppData\Local\Temp\ipykernel_24060\468073868.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Unnamed: 0', 1)


(467334, 39)


,S_SUM,U_L1_L2,I_SUM,F,U_L3_L1,C_phi_L1,U_L3_N,RealEc_L1,RealEc_L2,RealEc_L3,...,S_L3,Q_L1,S_L2,U_L2_N,S_L1,U_L1_N,I_L1,RealEc_SUM,I_L2,I_L3
ts,,,,,,,,,,,,,,,,,,,,,
2022-02-22T11:17:24.127703575Z,123259.65,411.39,1.12,50.01,411.12,0.99,236.57,31345348,30444914,30360338,...,42346.98,6085.66,39469.35,238.10,41443.33,237.23,174.70,92150704,165.77,179.00
2022-02-22T11:17:25.439348217Z,122868.31,411.33,1.05,50.01,409.30,0.98,236.51,31345362,30444928,30360352,...,40454.14,7272.30,39607.47,237.89,42806.71,236.40,181.08,92150752,166.49,171.04
2022-02-22T11:17:26.705083973Z,123108.94,411.02,1.04,50.01,410.57,0.98,237.13,31345374,30444940,30360364,...,40749.97,8029.02,41030.93,237.73,41328.03,237.20,174.23,92150784,172.60,171.85
2022-02-22T11:17:28.014494747Z,123048.30,410.90,1.02,50.01,410.75,0.99,236.20,31345388,30444952,30360380,...,42542.77,5632.11,39168.28,237.67,41337.26,237.06,174.37,92150824,164.80,180.11
2022-02-22T11:17:29.306443661Z,122208.24,409.81,1.08,50.01,409.76,0.99,235.70,31345400,30444964,30360392,...,41760.77,5949.70,39547.37,237.24,40900.10,236.52,172.93,92150864,166.70,177.18


In [4]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [5]:
history_window = 60 #seconds
prediction_window = 1
values = df.values
# frame as supervised learning
reframed = series_to_supervised(values, history_window, prediction_window)
# drop columns we don't want to predict
print(reframed.head())


   var1(t-5)  var2(t-5)  var3(t-5)  var4(t-5)  var5(t-5)  var6(t-5)  \
5  146233.91     409.18       1.01      50.03     406.46       0.98   
6  145337.03     408.08       1.24      50.03     406.35       0.99   
7  145837.16     408.78       0.85      50.03     408.23       0.99   
8  146147.23     409.27       1.00      50.03     407.24       0.99   
9  145390.61     406.74       1.24      50.03     406.39       0.99   

   var7(t-5)   var8(t-5)   var9(t-5)  var10(t-5)  ...  var30(t)  var31(t)  \
5     235.78  25869132.0  25165398.0  25027530.0  ...  45359.20  10621.60   
6     234.44  25869150.0  25165414.0  25027548.0  ...  49843.51   6159.55   
7     235.60  25869166.0  25165430.0  25027562.0  ...  48664.63   7762.82   
8     235.84  25869182.0  25165446.0  25027578.0  ...  46208.43   9556.19   
9     234.35  25869198.0  25165462.0  25027594.0  ...  49814.18   6721.20   

   var32(t)  var33(t)  var34(t)  var35(t)  var36(t)    var37(t)  var38(t)  \
5  50243.84    240.00  51408.31  

In [18]:
timesteps = prediction_window
features = (history_window + prediction_window) * 39

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import LayerNormalization

# design network
model = Sequential()
model.add(LayerNormalization(axis=0))
model.add(LSTM(100, input_shape=(timesteps, features)))
model.add(Dense(39))
model.compile(loss='mae', optimizer='adam')


In [20]:
# split into train and test sets
values = reframed.values
n_train_seconds = 2*24*60*60 #2 days
n_test_seconds =  3*24*60*60 #3 days
train = values[:n_train_seconds, :]
test = values[n_train_seconds:n_test_seconds, :]
# split into input and outputs
train_X, train_y = train[:, :-39], train[:, -39:]
test_X, test_y = test[:, :-39], test[:, -39:]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(172800, 1, 195) (172800, 39) (86400, 1, 195) (86400, 39)


In [21]:
history = model.fit(train_X, train_y, epochs=200, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)


Epoch 1/2
2400/2400 - 8s - loss: 12986233.0000 - val_loss: 13868734.0000 - 8s/epoch - 3ms/step
Epoch 2/2
2400/2400 - 6s - loss: 12986086.0000 - val_loss: 13868604.0000 - 6s/epoch - 3ms/step


In [25]:
history_results = pd.DataFrame([history.history['loss'], history.history['val_loss']], columns=['Loss', 'Validation Loss'])

In [27]:
history_results.to_csv('results/Simple_LSTM_60secs.csv')
model.save('models/Simple_LSTM_60secs')

INFO:tensorflow:Assets written to: models/Simple_LSTM_60secs\assets


INFO:tensorflow:Assets written to: models/Simple_LSTM_60secs\assets
